## Informations

This notebook only contains code with the pretrained YOLOv12 model on license plates detection.
It was used on Colab.

In [ ]:
import yaml


# Remplacez 'MonProjetYOLO' par le nom exact de votre dossier sur Google Drive
drive_path = '/content/drive/MyDrive/data'

data_config = {
    'path': drive_path,
    'train': 'images/train',
    'val': 'images/val',
    'names': {
        0: 'license_plate'
    }
}

with open('/content/license_plates.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

In [ ]:
from ultralytics import YOLO

# En appelant jus
# te le modèle, Ultralytics tente de le télécharger
# depuis leurs serveurs (GitHub Assets)
model = YOLO("yolo12n.pt")
print("Téléchargement réussi !")

In [ ]:
from ultralytics import YOLO

# Charger YOLOv12 Nano
model = YOLO('content/yolo12n.pt')

# Entraîner
results = model.train(
    data='/content/license_plates.yaml',
    epochs=20,
    imgsz=640,
    batch=32,
    device=0,
    fraction=0.075
)

In [4]:
from ultralytics import YOLO

model = YOLO('runs/detect/colab_train_Yolov12/weights/best.pt') # Ton modèle entraîné

# On lance la prédiction sur la vidéo
# save=True est l'argument clé pour générer le fichier vidéo
model.predict(source='../video/voiture_tristan.mp4', save=True, conf=0.5)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/151) c:\Users\celes\OneDrive\Documents\ENTPE\3A\ICVIS\ICVIS_project\notebooks\..\video\voiture_tristan.mp4: 640x384 (no detections), 367.7ms
video 1/1 (frame 2/151) c:\Users\celes\OneDrive\Documents\ENTPE\3A\ICVIS\ICVIS_project\notebooks\..\video\voiture_tristan.mp4: 640x384 (no detections), 223.7ms
video 1/1 (frame 3/151) c:\Users\celes\OneDrive\Documents\ENTPE\3A\ICVIS\ICVIS_project\notebooks\..\video\voiture_tristan.mp4: 640x384 1 li

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'license_plate'}
 obb: None
 orig_img: array([[[232, 220, 206],
         [232, 220, 206],
         [232, 220, 206],
         ...,
         [ 81,  85,  87],
         [ 81,  85,  87],
         [ 78,  82,  84]],
 
        [[232, 220, 206],
         [232, 220, 206],
         [232, 220, 206],
         ...,
         [ 80,  84,  86],
         [ 78,  82,  84],
         [ 78,  82,  84]],
 
        [[234, 222, 208],
         [234, 222, 208],
         [234, 222, 208],
         ...,
         [ 80,  84,  86],
         [ 78,  82,  84],
         [ 78,  82,  84]],
 
        ...,
 
        [[108, 111, 109],
         [115, 118, 116],
         [122, 125, 123],
         ...,
         [ 98,  99,  94],
         [ 88,  89,  84],
         [ 95,  96,  91]],
 
        [[116, 119, 117],
         [119, 122, 120],
         [122, 125, 123],
         ...,
         [1

In [ ]:
import cv2
import easyocr
from ultralytics import YOLO

# 1. Initialisation (Modèle + Lecteur OCR)
model = YOLO('runs/detect/colab_train_Yolov12/weights/best.pt')  # Votre modèle entraîné
reader = easyocr.Reader(['en']) # 'en' pour les caractères latins

video_path = '../video/voiture_tristan.mp4'
cap = cv2.VideoCapture(video_path)

# Préparation de l'écriture de la vidéo finale
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('../video/resultat_final.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 2. Détection avec YOLO
    results = model(frame, conf=0.5)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        # 3. Découpage de la plaque (Crop)
        plate_crop = frame[int(y1):int(y2), int(x1):int(x2)]

        if plate_crop.size > 0:
            # 4. Lecture de la plaque avec EasyOCR
            ocr_result = reader.readtext(plate_crop)
            
            if ocr_result:
                plate_text = ocr_result[0][-2].upper() # Récupère le texte en majuscules

                # 5. Affichage dans le coin supérieur gauche
                # Rectangle de fond pour la lisibilité
                cv2.rectangle(frame, (10, 10), (400, 80), (0, 0, 0), -1)
                # Texte de la plaque
                cv2.putText(frame, f"PLAQUE : {plate_text}", (20, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

                # Dessiner aussi le cadre autour de la plaque sur la voiture
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Affichage en direct (optionnel)
    # cv2.imshow('Detection et Lecture', frame)
    out.write(frame) # Sauvegarde chaque image dans la vidéo finale

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Vidéo terminée et enregistrée sous '../video/resultat_final.mp4'")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 640x384 (no detections), 523.6ms
Speed: 14.0ms preprocess, 523.6ms inference, 5.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 86.4ms
Speed: 3.8ms preprocess, 86.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 127.3ms
Speed: 3.0ms preprocess, 127.3ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 75.3ms
Speed: 2.8ms preprocess, 75.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 115.1ms
Speed: 4.3ms preprocess, 115.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 156.4ms
Speed: 6.3ms preprocess, 156.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 98.7ms
Speed: 3.5ms preprocess, 98.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 100.8ms
Speed: 3.7ms preprocess

In [4]:
import cv2
import easyocr
import os
from ultralytics import YOLO
from collections import Counter

# 1. Chargement des modèles
model = YOLO('runs/detect/colab_train_Yolov12/weights/best.pt')  # Assurez-vous que le fichier est dans le même dossier
reader = easyocr.Reader(['en'], gpu=True) # Utilise le GPU si disponible

# 2. Configuration Vidéo
video_path = '../video/voiture_tristan.mp4'
if not os.path.exists(video_path):
    print(f"Erreur : Le fichier {video_path} est introuvable.")
    exit()

cap = cv2.VideoCapture(video_path)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = int(cap.get(cv2.CAP_PROP_FPS))

# Préparation de la vidéo de sortie
out = cv2.VideoWriter('../video/resultat_final_stable.mp4', 
                       cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# --- VARIABLES DE STABILISATION ---
plate_history = []     # Liste pour stocker les dernières lectures
max_history = 30       # Nombre de frames à mémoriser pour le "vote"
stable_text = "En attente..."

print("Analyse en cours... Appuyez sur 'q' pour arrêter.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 3. Détection de la plaque avec YOLO
    results = model(frame, conf=0.5, verbose=False)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        
        # Découpage (Crop) de la zone de la plaque
        plate_img = frame[int(y1):int(y2), int(x1):int(x2)]
        
        if plate_img.size > 0:
            # 4. Lecture OCR
            ocr_results = reader.readtext(plate_img)
            
            if ocr_results:
                # Récupère le texte, le nettoie et le met en majuscules
                current_read = ocr_results[0][-2].upper().strip()
                
                # Ajout à l'historique pour stabilisation
                plate_history.append(current_read)
                if len(plate_history) > max_history:
                    plate_history.pop(0)

                # 5. SYSTÈME DE VOTE : On prend le texte le plus fréquent
                stable_text = Counter(plate_history).most_common(1)[0][0]

        # Dessiner le cadre vert sur la voiture
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # 6. AFFICHAGE DANS LE COIN SUPÉRIEUR GAUCHE
    # Fond noir pour la lisibilité
    cv2.rectangle(frame, (0, 0), (600, 100), (0, 0, 0), -1)
    # Texte stabilisé
    cv2.putText(frame, f"PLAQUE : {stable_text}", (20, 65), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 255, 0), 4)

    # 7. Sauvegarde et Visualisation
    out.write(frame)
    
    # NOTE : Si cv2.imshow plante encore, commentez les deux lignes suivantes
    cv2.imshow('Detection de plaque stabilisée', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Traitement terminé. Vidéo enregistrée sous 'resultat_final_stable.mp4'")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Analyse en cours... Appuyez sur 'q' pour arrêter.
Traitement terminé. Vidéo enregistrée sous 'resultat_final_stable.mp4'
